# Importando bibliotecas

In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import timedelta

# Conectando com o servidor SQL e realizando a consulta

In [2]:
# Dados de conexao
server = 'localhost'
database = 'AdventureWorksDW2022'

# String de conexão para autenticação integrada
connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'

# Criando o engine de conexão
engine = create_engine(connection_string)

# Script SQL
sql_script = '''
SELECT
	FIS.SalesOrderNumber,
	FIS.ProductKey,
	PD.EnglishProductName,
	PS.ProductSubcategoryKey,
	PS.EnglishProductSubcategoryName,
	FIS.OrderDate,
	FIS.CustomerKey,
	CT.FirstName,
	FIS.SalesAmount
FROM FactInternetSales AS FIS
LEFT JOIN DimProduct AS PD ON (PD.ProductKey = FIS.ProductKey)
LEFT JOIN DimCustomer AS CT ON (CT.CustomerKey = FIS.CustomerKey)
LEFT JOIN DimProductSubcategory AS PS ON (PS.ProductSubcategoryKey = PD.ProductSubcategoryKey)
'''

# Criando um objeto TextClause
query = text(sql_script)

# Criando o Dataframe
df = pd.read_sql_query(query, engine)

df.head(3)

,SalesOrderNumber,ProductKey,EnglishProductName,ProductSubcategoryKey,EnglishProductSubcategoryName,OrderDate,CustomerKey,FirstName,SalesAmount
0,SO43697,310,"Road-150 Red, 62",2,Road Bikes,2010-12-29,21768,Cole,3578.27
1,SO43698,346,"Mountain-100 Silver, 44",1,Mountain Bikes,2010-12-29,28389,Rachael,3399.99
2,SO43699,346,"Mountain-100 Silver, 44",1,Mountain Bikes,2010-12-29,25863,Sydney,3399.99


# Criando a variável para leitura do Excel

In [3]:
# Caminho arquivo
path = r'C:\Automação Cashback\\'

# Nome Arquivo
arquivo = '% Cashback.xlsx'

# Lendo o arquivo com os valores mínimos e percentuais de cashback
arquivo_cashback = pd.read_excel(f'{path}{arquivo}')

arquivo_cashback.head(3)

,ProductSubcategoryKey,EnglishProductSubcategoryName,Price Limite,%CashBack
0,26,Bike Racks,130,0.05
1,27,Bike Stands,160,0.05
2,28,Bottles and Cages,10,0.10


# Tratando o Dataframe gerado pelo Banco SQL

### Ajustando o DataFrame para obter dados dos últimos 90 dias

In [4]:
# Verificando a data maxima 

max(df['OrderDate'])

Timestamp('2014-01-28 00:00:00')

In [5]:
# Filtrando a base nos ultimos 90 dias

# Criando uma variavel com a data de 90 dias atras
dias = 90

ultimo_dias = df['OrderDate'].max() - timedelta(days=dias)

# Filtrando o Dataframe
df_ultimos_dias = df.query('OrderDate >= @ultimo_dias')

# Conferindo a data minima
min(df_ultimos_dias['OrderDate'])

Timestamp('2013-10-30 00:00:00')

### Extraindo somente as colunas necessarias

In [6]:
# Selecionando colunas
colunas = [
    'ProductSubcategoryKey',
    'EnglishProductSubcategoryName',
    'CustomerKey',
    'FirstName',
    'SalesAmount'
]

df_tratado = df_ultimos_dias[colunas]

df_tratado.head(2)

,ProductSubcategoryKey,EnglishProductSubcategoryName,CustomerKey,FirstName,SalesAmount
47349,28,Bottles and Cages,11974,Randy,8.99
47350,28,Bottles and Cages,11974,Randy,4.99


### Realizando um agrupamento dos dados 

In [7]:
# Agrupando os dados

df_sql_consolidado = df_ultimos_dias.groupby([
    'ProductSubcategoryKey','EnglishProductSubcategoryName','CustomerKey','FirstName'
    ])['SalesAmount'].sum().reset_index()


df_sql_consolidado.head(3)

,ProductSubcategoryKey,EnglishProductSubcategoryName,CustomerKey,FirstName,SalesAmount
0,1,Mountain Bikes,11044,Adam,2294.99
1,1,Mountain Bikes,11046,Christine,2319.99
2,1,Mountain Bikes,11047,Jaclyn,2319.99


# Agrupando o DF do BANCO com o do EXCEL

In [8]:
# Realizando o agrupamento dos dados

df_agrupado = pd.merge(
    df_sql_consolidado,
    arquivo_cashback[['ProductSubcategoryKey','Price Limite', '%CashBack']],
    how ='left',
    on ='ProductSubcategoryKey'
)

df_agrupado.head(2)

,ProductSubcategoryKey,EnglishProductSubcategoryName,CustomerKey,FirstName,SalesAmount,Price Limite,%CashBack
0,1,Mountain Bikes,11044,Adam,2294.99,800,0.07
1,1,Mountain Bikes,11046,Christine,2319.99,800,0.07


# Realizando a validação e o calculo do CASHBACK

In [9]:
# Criando a coluna de validação e calculo

df_agrupado['Cashback'] = df_agrupado.apply(
    lambda row: row['SalesAmount'] * row['%CashBack'] if row['SalesAmount'] > row['Price Limite']
    else 0,
    axis = 1
)

df_agrupado.head(3)

,ProductSubcategoryKey,EnglishProductSubcategoryName,CustomerKey,FirstName,SalesAmount,Price Limite,%CashBack,Cashback
0,1,Mountain Bikes,11044,Adam,2294.99,800,0.07,160.6493
1,1,Mountain Bikes,11046,Christine,2319.99,800,0.07,162.3993
2,1,Mountain Bikes,11047,Jaclyn,2319.99,800,0.07,162.3993


# Criando um Datrafame consolidado com os Customers e seu respectivo Cashback

In [10]:
# Filtrando somente as colunas necessarias
df_colunas = [
    'CustomerKey',
    'FirstName',
    'Cashback'
]

df_cashback = df_agrupado[df_colunas]


# Realizando a consolidação
df_cashback = df_cashback.groupby(['CustomerKey','FirstName'])['Cashback'].sum().reset_index()

# Removendo cashback = 0
df_cashback = df_cashback.query('Cashback > 0')

df_cashback.head(3)

,CustomerKey,FirstName,Cashback
0,11001,Eugene,1.3980
1,11013,Ian,5.9985
2,11019,Luke,15.9425


# Salvando o Dataframe em EXCEL com os Customers e seus Cashbacks

In [11]:
# Caminho onde arquivo vai salvar
path = r'C:\Automação Cashback\\'
nome = 'Clientes e Cashback.xlsx'

try:
    df_cashback.to_excel(f'{path}{nome}', sheet_name='Cashback', index=False)
    print('Arquivo salvo com sucesso!')
    
except Exception as e:
    print(f"Ocorreu um erro: {e}")


Arquivo salvo com sucesso!
